In [ ]:
# Gerekli kütüphaneleri yükle
!pip install pandas numpy scikit-learn scipy matplotlib seaborn -q


## 1. Veri Yükleme ve Hazırlama

Bu hücre, MotoGP verilerini yükler veya kaydedilmiş veriyi kullanır.


In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Veri klasörü
DATA_DIR = Path("ml_data")
DATA_DIR.mkdir(exist_ok=True)

def load_motogp_data():
    """
    MotoGP verilerini yükle veya kaydedilmiş veriyi kullan
    """
    cache_file = DATA_DIR / "motogp_full_cache.pkl"
    
    # Kaydedilmiş veri varsa yükle
    if cache_file.exists():
        print("✅ Kaydedilmiş MotoGP verisi yükleniyor...")
        with open(cache_file, 'rb') as f:
            data = pickle.load(f)
        print(f"   Veri yüklendi: {len(data)} satır, {len(data.columns)} kolon")
        print(f"   Kolonlar: {list(data.columns[:10])}...")
        return data
    
    # Ana notebook'tan CSV olarak yükle (eğer varsa)
    csv_file = Path("trends_data") / "motogp_full_data.csv"
    if csv_file.exists():
        print("✅ CSV dosyasından MotoGP verisi yükleniyor...")
        data = pd.read_csv(csv_file)
        if 'RaceDate' in data.columns:
            data['RaceDate'] = pd.to_datetime(data['RaceDate'])
        print(f"   Veri yüklendi: {len(data)} satır")
        # Kaydet
        with open(cache_file, 'wb') as f:
            pickle.dump(data, f)
        print(f"   Veri kaydedildi: {cache_file}")
        return data
    
    # Eğer ana notebook'ta motogp_full değişkeni varsa (Jupyter kernel'da)
    try:
        # Jupyter/IPython ortamında çalışıyorsak, ana notebook'tan değişkeni al
        import sys
        if 'ipykernel' in sys.modules or 'IPython' in sys.modules:
            # IPython namespace'inden al
            try:
                from IPython import get_ipython
                ipython = get_ipython()
                if ipython is not None and 'motogp_full' in ipython.user_ns:
                    print("✅ Ana notebook'tan motogp_full değişkeni yükleniyor...")
                    data = ipython.user_ns['motogp_full'].copy()
                    print(f"   Veri yüklendi: {len(data)} satır")
                    # Kaydet
                    with open(cache_file, 'wb') as f:
                        pickle.dump(data, f)
                    print(f"   Veri kaydedildi: {cache_file}")
                    return data
            except:
                pass
    except:
        pass
    
    print("⚠️  Kaydedilmiş veri bulunamadı.")
    print("   Lütfen önce ana analiz notebook'unu çalıştırıp motogp_full verisini oluşturun.")
    print("   Veya aşağıdaki hücrede veriyi manuel olarak yükleyin.")
    return None

# Veriyi yükle
motogp_data = load_motogp_data()

# Veri yüklendiyse önizleme göster
if motogp_data is not None:
    print(f"\n📊 Veri Önizleme:")
    print(f"   Shape: {motogp_data.shape}")
    print(f"   Kolonlar ({len(motogp_data.columns)}):")
    for i, col in enumerate(motogp_data.columns):
        print(f"     {i+1}. {col}")
    print(f"\n   İlk 5 satır:")
    print(motogp_data.head())


### Veriyi Hazırlama ve Özellik Mühendisliği

Hedef değişken: **Relative Increase %** kategorilerine göre sınıflandırma
- **Low**: < 0% (azalma veya değişim yok)
- **Medium**: 0-50% (orta artış)
- **High**: > 50% (yüksek artış)


In [ ]:
def prepare_ml_data(df):
    """
    MotoGP verisini ML için hazırla
    
    Args:
        df: motogp_full DataFrame
    
    Returns:
        X: Feature matrix
        y: Target labels
        feature_names: Feature isimleri
    """
    # Veriyi kopyala
    data = df.copy()
    
    # NaN değerleri temizle
    data = data.dropna(subset=['Relative Increase %', 'Popularity', 
                               'Career wins_num', 'Career podiums_num', 
                               'Championships_num'])
    
    # Sonsuz değerleri temizle
    data = data[np.isfinite(data['Relative Increase %'])]
    
    # Hedef değişkeni kategorilere ayır
    def categorize_increase(increase):
        if pd.isna(increase) or not np.isfinite(increase):
            return None
        if increase < 0:
            return 'Low'
        elif increase < 50:
            return 'Medium'
        else:
            return 'High'
    
    data['target_category'] = data['Relative Increase %'].apply(categorize_increase)
    data = data.dropna(subset=['target_category'])
    
    # Özellikleri seç
    feature_cols = [
        'Popularity',
        'Career wins_num',
        'Career podiums_num',
        'Championships_num',
        'Years_active_len',
        'Search Before',
        'Trend Difference'
    ]
    
    # Sadece mevcut kolonları kullan
    available_features = [col for col in feature_cols if col in data.columns]
    
    X = data[available_features].values
    y = data['target_category'].values
    
    # Label encoding
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    
    print(f"✅ Veri hazırlandı:")
    print(f"   Örnek sayısı: {len(X)}")
    print(f"   Özellik sayısı: {X.shape[1]}")
    print(f"   Özellikler: {available_features}")
    print(f"   Sınıf dağılımı:")
    for label, count in zip(le.classes_, np.bincount(y_encoded)):
        print(f"     {label}: {count}")
    
    return X, y_encoded, available_features, le

# Veriyi hazırla (eğer motogp_data varsa)
if motogp_data is not None:
    X, y, feature_names, label_encoder = prepare_ml_data(motogp_data)
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Ölçeklendirme
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"\n✅ Train/Test split yapıldı:")
    print(f"   Train: {X_train.shape[0]} örnek")
    print(f"   Test: {X_test.shape[0]} örnek")
else:
    print("⚠️  Veri yüklenemedi. Lütfen önce veriyi yükleyin.")


## 2. Random Forest Classifier

Ana sınıflandırıcı model - MotoGP fan reaction kategorilerini tahmin eder.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

def train_random_forest(X_train, X_test, y_train, y_test, 
                        n_estimators=100, max_depth=None, random_state=42):
    """
    Random Forest modelini eğit
    
    Args:
        X_train, X_test: Eğitim ve test özellikleri
        y_train, y_test: Eğitim ve test etiketleri
        n_estimators: Ağaç sayısı
        max_depth: Maksimum derinlik
        random_state: Rastgelelik seed'i
    
    Returns:
        model: Eğitilmiş model
    """
    print("🌲 Random Forest eğitiliyor...")
    print(f"   Parametreler: n_estimators={n_estimators}, max_depth={max_depth}")
    
    # Model oluştur
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=random_state,
        n_jobs=-1,
        class_weight='balanced'  # Sınıf dengesizliğini dikkate al
    )
    
    # Eğit
    model.fit(X_train, y_train)
    
    # Tahmin
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Değerlendirme
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    
    print(f"\n✅ Model eğitildi!")
    print(f"   Train Accuracy: {train_acc:.4f}")
    print(f"   Test Accuracy: {test_acc:.4f}")
    
    # Özellik önemleri
    feature_importance = pd.DataFrame({
        'feature': feature_names,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\n📊 Özellik Önemleri:")
    print(feature_importance.to_string(index=False))
    
    # Classification report
    print(f"\n📊 Classification Report (Test):")
    print(classification_report(y_test, y_pred_test, 
                                target_names=label_encoder.classes_))
    
    # Modeli kaydet
    model_file = DATA_DIR / "random_forest_model.pkl"
    with open(model_file, 'wb') as f:
        pickle.dump({
            'model': model,
            'scaler': scaler,
            'label_encoder': label_encoder,
            'feature_names': feature_names
        }, f)
    print(f"\n💾 Model kaydedildi: {model_file}")
    
    return model, feature_importance

# Modeli eğit (eğer veri hazırsa)
if 'X_train_scaled' in locals() and 'X_test_scaled' in locals():
    rf_model, feature_importance = train_random_forest(
        X_train_scaled, X_test_scaled, y_train, y_test,
        n_estimators=100, max_depth=10
    )
else:
    print("⚠️  Veri hazır değil. Lütfen önce veri hazırlama hücresini çalıştırın.")


## 3. Decision Tree Classifier

Baseline model - Raporda açıklama ve karşılaştırma için kullanılır.


In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

def train_decision_tree(X_train, X_test, y_train, y_test,
                       max_depth=5, min_samples_split=10, random_state=42):
    """
    Decision Tree modelini eğit (baseline)
    
    Args:
        X_train, X_test: Eğitim ve test özellikleri
        y_train, y_test: Eğitim ve test etiketleri
        max_depth: Maksimum derinlik
        min_samples_split: Split için minimum örnek sayısı
        random_state: Rastgelelik seed'i
    
    Returns:
        model: Eğitilmiş model
    """
    print("🌳 Decision Tree eğitiliyor...")
    print(f"   Parametreler: max_depth={max_depth}, min_samples_split={min_samples_split}")
    
    # Model oluştur
    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=random_state,
        class_weight='balanced'
    )
    
    # Eğit
    model.fit(X_train, y_train)
    
    # Tahmin
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Değerlendirme
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    
    print(f"\n✅ Model eğitildi!")
    print(f"   Train Accuracy: {train_acc:.4f}")
    print(f"   Test Accuracy: {test_acc:.4f}")
    
    # Classification report
    print(f"\n📊 Classification Report (Test):")
    print(classification_report(y_test, y_pred_test,
                                target_names=label_encoder.classes_))
    
    # Ağaç görselleştirme (küçük derinlik için)
    if max_depth <= 5:
        plt.figure(figsize=(20, 10))
        plot_tree(model, 
                  feature_names=feature_names,
                  class_names=label_encoder.classes_,
                  filled=True,
                  rounded=True,
                  fontsize=10)
        plt.title("Decision Tree Visualization", fontsize=16)
        plt.tight_layout()
        plt.savefig(DATA_DIR / "decision_tree_visualization.png", dpi=300, bbox_inches='tight')
        print(f"\n📊 Ağaç görselleştirmesi kaydedildi: {DATA_DIR / 'decision_tree_visualization.png'}")
        plt.show()
    
    # Modeli kaydet
    model_file = DATA_DIR / "decision_tree_model.pkl"
    with open(model_file, 'wb') as f:
        pickle.dump({
            'model': model,
            'scaler': scaler,
            'label_encoder': label_encoder,
            'feature_names': feature_names
        }, f)
    print(f"\n💾 Model kaydedildi: {model_file}")
    
    return model

# Modeli eğit (eğer veri hazırsa)
if 'X_train_scaled' in locals() and 'X_test_scaled' in locals():
    dt_model = train_decision_tree(
        X_train_scaled, X_test_scaled, y_train, y_test,
        max_depth=5, min_samples_split=10
    )
else:
    print("⚠️  Veri hazır değil. Lütfen önce veri hazırlama hücresini çalıştırın.")


## 4. Model Karşılaştırması

Random Forest ve Decision Tree modellerinin performans karşılaştırması


In [ ]:
def compare_models(rf_model, dt_model, X_test, y_test):
    """
    İki modeli karşılaştır
    """
    # Tahminler
    rf_pred = rf_model.predict(X_test)
    dt_pred = dt_model.predict(X_test)
    
    # Accuracy
    rf_acc = accuracy_score(y_test, rf_pred)
    dt_acc = accuracy_score(y_test, dt_pred)
    
    print("=" * 60)
    print("MODEL KARŞILAŞTIRMASI")
    print("=" * 60)
    print(f"\n📊 Test Accuracy:")
    print(f"   Random Forest: {rf_acc:.4f}")
    print(f"   Decision Tree: {dt_acc:.4f}")
    print(f"   Fark: {abs(rf_acc - dt_acc):.4f}")
    
    # Confusion matrices
    print(f"\n📊 Confusion Matrices:")
    print(f"\nRandom Forest:")
    print(confusion_matrix(y_test, rf_pred))
    print(f"\nDecision Tree:")
    print(confusion_matrix(y_test, dt_pred))
    
    # Karşılaştırma DataFrame
    comparison = pd.DataFrame({
        'Model': ['Random Forest', 'Decision Tree'],
        'Accuracy': [rf_acc, dt_acc],
        'Difference': [rf_acc - dt_acc, dt_acc - rf_acc]
    })
    
    print(f"\n📊 Karşılaştırma Tablosu:")
    print(comparison.to_string(index=False))
    
    return comparison

# Modelleri karşılaştır (eğer her ikisi de eğitildiyse)
if 'rf_model' in locals() and 'dt_model' in locals():
    comparison_df = compare_models(rf_model, dt_model, X_test_scaled, y_test)
else:
    print("⚠️  Her iki model de eğitilmedi. Lütfen önce modelleri eğitin.")
